In [1]:
from typing import Dict
from rich import print
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#custom imports
from Load import Load_Type, Load_Category, Load, Wall, Concrete, Soil

In [4]:
#Soil Properties
gamma_s = 19.0 # kPa/m
ka = 0.33
ko = 0
kp = 3.0
Q_uls = 300
Q_sls = 200
mu = 0.4

soil = Soil('soil', gamma_s, ka, ko, kp, Q_uls, Q_sls, mu)

# concrete properties
fc = 25 # MPa
Ec = 4500*(fc**0.5) # MPa
alpha1 = 0.91
beta1 = 0.81

C25 = Concrete('C25', fc)

In [5]:
# Geometry
# Wall
h = 1800 # mm
tw = 200 # mm
L = 1000 # mm

# Footing
tc = 300 # mm
toe = 250 # mm
b = 1500 # mm
D = 300 # mm
heel = b-toe-tw # mm

wall = Wall('wall1', h, tw, toe, b, D, heel, C25)

In [54]:
# Loads
# Soil
q_soil = ka*gamma_s*((h+tc+D)/1000)*(L/1000) # kPa
q_eq = 0.25*gamma_s*((h+tc+D)/1000)*(L/1000) # kPa

# compaction
q_comp = 5 # kPa

# surcharge
W_dl = 2.4 # kPa
W_ll = 4.8 # kpa

# Wall Loads
P_dl = 100 # kN/m
P_sl = 200 # kN/m

wall.Add_Load(Load_Type.HYDROSTATIC, Load_Category.SOIL, 'soil', q_soil)
wall.Add_Load(Load_Type.HYDROSTATIC, Load_Category.SEISMIC, 'eq', q_eq)

wall.Add_Load(Load_Type.SURCHARGE, Load_Category.COMPACTION, 'comp', q_comp)

wall.Add_Load(Load_Type.SURCHARGE, Load_Category.DEAD, 'dl', W_dl)
wall.Add_Load(Load_Type.SURCHARGE, Load_Category.LIVE, 'll', W_ll)

wall.Add_Load(Load_Type.WALL, Load_Category.DEAD, 'wall_dl', P_dl)
wall.Add_Load(Load_Type.WALL, Load_Category.SNOW, 'wall_sl', P_sl)

In [55]:
for k, load in wall.Loads.items():
    pass
    #print(load.Category)
    #print(load.Type)

In [68]:
def Sum_Loads(loads: Dict, load_type: Load_Type)-> Dict:
    load_dict = {
        'Dead': 0,
        'Live': 0,
        'Storage': 0,
        'Snow': 0,
        'Seismic': 0,
        'Soil': 0,
        'Compaction': 0
    }
    for k, load in loads.items():
        if load.Type == load_type:
            if load.Category == Load_Category.DEAD:
                load_dict['Dead'] =+ load.Value
            if load.Category == Load_Category.LIVE:
                load_dict['Live'] =+ load.Value
            if load.Category == Load_Category.STORAGE_LIVE:
                load_dict['Storage'] =+ load.Value
            if load.Category == Load_Category.SNOW:
                load_dict['Snow'] =+ load.Value
            if load.Category == Load_Category.SEISMIC:
                load_dict['Seismic'] =+ load.Value
            if load.Category == Load_Category.SOIL:
                load_dict['Soil'] =+ load.Value
            if load.Category == Load_Category.COMPACTION:
                load_dict['Compatcion'] =+ load.Value
    return load_dict

def Load_Combos(load_dict: Dict) -> Dict:
    combo_dict = {}
    combo_dict['1.4DL'] = load_dict['Dead'] * 1.4
    combo_dict['1.25DL+1.5LL+1.0SL'] = load_dict['Dead'] * 1.25 + load_dict['Live'] * 1.5 + load_dict['Snow'] * 1.0
    combo_dict['1.25DL+1.5SL+1.0LL'] = load_dict['Dead'] * 1.25 + load_dict['Snow'] * 1.5 + load_dict['Live'] * 1.0
    combo_dict['1.25DL+1.5LL'] = load_dict['Dead'] * 1.25 + load_dict['Live'] * 1.5 
    combo_dict['1.25DL+1.5SL'] = load_dict['Dead'] * 1.25 + load_dict['Snow'] * 1.5 
    # combo_df = pd.DataFrame(combo_dict)
    return combo_dict

In [74]:
hydro_loads = {}
sur_loads = {}
wall_loads = {}

hydro_loads = Sum_Loads(wall.Loads, Load_Type.HYDROSTATIC)
sur_loads = Sum_Loads(wall.Loads, Load_Type.SURCHARGE)
wall_loads = Sum_Loads(wall.Loads, Load_Type.WALL)

hydro_combos = Load_Combos(hydro_loads)
sur_combos = Load_Combos(sur_loads)
wall_combos = Load_Combos(wall_loads)

print(wall_combos)

{
    '1.4DL': 140.0,
    '1.25DL+1.5LL+1.0SL': 325.0,
    '1.25DL+1.5SL+1.0LL': 425.0,
    '1.25DL+1.5LL': 125.0,
    '1.25DL+1.5SL': 425.0
}

In [ ]:
# load cases
loads = {}
# Construction
q_soil
q_comp
blend = True
if blend:
    x = q_comp/(ka*gamma_s*(L/1000))*1000
    if x > (h+tc+D):
        x = 0
P_comp = q_comp*(h+tc+D)/1000*L/1000
P_soil = q_soil*x/2000

V = P_comp + P_soil
M = P_comp*(h+tc+D)/2000 + P_soil * (h+tc+D)/3000
geo_loads['LC1'] = [V,M]

# Service
q_soil
q_comp
blend = True
if blend:
    x = q_comp/(ka*gamma_s*(L/1000))*1000
    if x > (h+tc+D):
        x = 0
P_comp = q_comp*(h+tc+D)/1000*L/1000
P_soil = q_soil*x/2000

P = P_comp + P_soil
M = P_comp*(h+tc+D)/2000 + P_soil * (h+tc+D)/3000
loads['LC1'] = [P,M]